In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

from pinecone import Pinecone, ServerlessSpec

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


C:\Users\lenovo\.conda\envs\mchatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY='89445412-091e-44b0-9b54-96a56c9e0476'
#!export PINECONE_API_KEY
import os
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY

In [3]:
## extract data from the pdf

def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents=loader.load() 

    return documents

In [6]:
#!pip install pypdf

In [6]:
extracted_data=load_pdf("data/")

In [7]:
extracted_data[100] #sample data

Document(page_content='scores are uniformly distributed, we might then suppose that their average is\nalso uniformly distributed. \nBut is this actually true? \nBegin by considering the\nextremes: there is only one way to obtain a mean test score of 300; both\nindividuals must score 300. \nSimilarly, to obtain a mean of 0, both individuals\nmust score 0. \nBy contrast, consider a mean of 150. \nThis could result from a\nnumber of individual score combinations, for example (\nscore\nA\n, \nscore\nB\n) = :\n(150,150),(100,200),(125,175). \nIntuitively, there are many more ways to\nobtain moderate values for the sample mean than there are for the extremes.\nThis central tendency of the sample mean increases along with sample size,\nsince extreme values then require more individual scores to be\nsimultaneously extreme, which is less likely. \nThis effect is visible in \nFigure\n3.15\n; however, we also see another impact on the probability distribution for\nthe mean: as our sample size inc

In [8]:
#Create text chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20) ## overlapp b/w embedddings
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("length of chunks: ",len(text_chunks))

length of chunks:  2443


In [10]:
#download embedding model
def download_hf_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [5]:
embeddings=download_hf_embeddings()

In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [11]:
query_result=embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [12]:
## Initializing a given pinecone index/knowledge base
index_name="mchatbot"
#docsearch=PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)
docs_chunks =[t.page_content for t in text_chunks]
docsearch=PineconeVectorStore.from_texts(docs_chunks ,embeddings, index_name=index_name)

In [18]:
query = "What is Bayesian inference?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

3.7 The Bayesian Inference Process from
the Bayesian Formula
In Bayesian statistics, we use probability distributions to describe all
components of our system. 
Our starting point is Bayes’ rule (for a continuous
random variable):
In statistics, we aim to estimate a number of parameters, which from now on
we call 
θ
. 
We can think of these parameters as levers that we can pull to
change the behaviour of a statistical model. 
These parameters can be thought


In [19]:
prompt_template=""" 
Use the following pieces of information to answer the user's question.
If you don't know the answer, just state that you don't know, don't try to make up an answer.


Context:{context}
Question:{question}

Only return the helpful answer below and nothing else.

Helpful answer: 
"""

In [20]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [21]:
llm=CTransformers(model="model/llama-7b.ggmlv3.q4_1.bin",
                 model_type="llama",
                 config={'max_new_tokens':512,
                        'temperature':0.8})

In [27]:

qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    chain_type_kwargs=chain_type_kwargs)

# retriever = docsearch.as_retriever(search_kwargs={'k':2})
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",  # Use an appropriate chain type
#     retriever=retriever,
#     chain_type_kwargs=chain_type_kwargs
# )

In [28]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query":user_input})
    print("Response : ",result["result"])

C:\Users\lenovo\.conda\envs\mchatbot\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Bayes' Theorem says the conditional probability of an event A given that B has occurred, or the probability P(A | B), where A and B are random variables whose values are determined by some unknown probability distribution with density p(x) and q(x). 
In the general formulation, we have:

Bayes' theorem
```
P(A|B) = \frac{P(B|A) P(A)}{P(B)} .
```

If we consider conditional probabilities under a uniform distribution (that is, if p(x) and q(x) are equal to constant 1/n), the expression can be simplified:

\begin{code}

P(A|B) = \frac{P(B|A)}{P(B)} ,
\end{code}
which is known as Bayes' rule. 
The rule gives the probability of event A given knowledge about B, which we know through B) (h 
can interpret asume from now call prioritgained at least commonly abbrevi assume to use it 
call theor a priori can also refer to derive using B 
usually means 
refer to denote P(a are allowed usu have denoted with pigthinknow know is called P(or aquired before knew already know ascribe ascerta

KeyboardInterrupt: 